In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

data = pd.read_csv("/content/london_flows.csv")




data0=data

for index, row in data.iterrows():
    if row["flows"]==0 or row["distance"]==0:
        data.drop(index, inplace=True)

Oi = data["population"]
Dj = data["jobs"]
dist = data["distance"]
Tij=data["flows"]



import scipy.stats

def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2



from math import sqrt

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE



import numpy as np
#subset the dataframe to the flows we want
data_flows = data[["flows", "distance"]]
#remove all 0 values (logarithms can't deal with 0 values)
data_flows = data_flows[(data_flows!=0).all(1)]



#extract the x and y converting to log
x = np.log(data_flows["distance"])
y = np.log(data_flows["flows"])



import statsmodels.api as sm
import statsmodels.formula.api as smf

#take the variables and produce logarithms of them
x_variables = ["population","jobs","distance",]
log_x_vars = []
for x in x_variables:
    data[f"log_{x}"] = np.log(data[x])
    log_x_vars.append(f"log_{x}")

formula = 'flows ~  log_population + log_jobs + log_distance '

#run the regression
uncosim = smf.glm(formula = formula, 
                  data=data, 
                  family=sm.families.Poisson()).fit()



#extract the summary of the constrained model
print(uncosim.summary())



#first assign the parameter values from the model to the appropriate variables
K = uncosim.params[0]
alpha = uncosim.params[1]
gamma = uncosim.params[2]
beta = -uncosim.params[3]

data["unconstrainedEst2"] = np.exp(K 
                                       + alpha*data["log_population"] 
                                       + gamma*data["log_jobs"] 
                                       + beta*data["log_distance"])

#or we can just extract the results from the actual poisson regression and apply them to the data
predictions = uncosim.get_prediction()
predictions_summary_frame = predictions.summary_frame()
data["fitted"] = predictions_summary_frame["mean"]



#round the numbers so that we don't get a half of a person
data["unconstrainedEst2"] = round(data["unconstrainedEst2"], 0)
#convert to integers
data["unconstrainedEst2"] = data["unconstrainedEst2"].astype(int)
#check that the sum of these estimates make sense
sum(data["unconstrainedEst2"])



CalcRSqaure=CalcRSqaured(data["flows"], data["unconstrainedEst2"])
print("CalcRSqaure is",CalcRSqaure)



CalcRMSE_value=CalcRMSE(data["flows"], data["unconstrainedEst2"])
print("CalcRMSE is",CalcRMSE_value)
exit()


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                43945
Model:                            GLM   Df Residuals:                    43941
Model Family:                 Poisson   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1675e+06
Date:                Sun, 08 May 2022   Deviance:                   2.1627e+06
Time:                        20:18:15   Pearson chi2:                 4.13e+06
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -3.0646      0.014   -223.